# lang-24 Debug Report

## Set projectName and projectID

In [1]:
var projectName = "lang";
var projectId = "24";

## Set project (Do NOT change below cell)

In [2]:
var project = projectName + " " + projectId;
var jisdCmd = "jisd-d4j"// "/defects4j/framework/bin/jisd-d4j"; //(If you use VSCode)

In [3]:
exec(jisdCmd+" build "+project)

Running ant (compile)...................................................... OK
Running ant (compile.tests)................................................ OK


Optional[[Ljava.lang.String;@3385106]

In [4]:
exec(jisdCmd+" test "+project)

JUnit version 4.8.2
...................................................
Time: 0.038

OK (51 tests)


Running ant (compile)...................................................... OK
Running ant (compile.tests)................................................ OK


Optional[[Ljava.lang.String;@23092679]

## Create Debugger and run

### Get propaties of this project

In [5]:
var propKey = new ArrayList<>(Arrays.asList(
    "projdir", "srcdir", "testdir", "testclass", "testmethod", "line"));
var propValue = propKey.stream().map(k->exec(jisdCmd+" "+k+" "+project).get()[0]).collect(Collectors.toList());
var prop = new HashMap<String, String>();
for(int i = 0; i < propKey.size(); i++) {
    prop.put(propKey.get(i), propValue.get(i));
}
// Properties for later use
var projectDir = prop.get("projdir");
var mainSrcDir = prop.get("srcdir");
var testSrcDir = prop.get("testdir");
var testClass  = prop.get("testclass");
var testMethod = prop.get("testmethod");
var failedLine = Integer.parseInt(prop.get("line"));

/defects4j/tmp/lang_24_buggy
/defects4j/tmp/lang_24_buggy/src/main/java
/defects4j/tmp/lang_24_buggy/src/test/java
org.apache.commons.lang3.math.NumberUtilsTest
testIsNumber
1145


### Run output cell below from Terminal

In [6]:
println(jisdCmd+" debug "+project)

jisd-d4j debug lang 24


### Connect to the target VM

In [7]:
var dbg = new Debugger(25432);
dbg.setSrcDir(mainSrcDir, testSrcDir);
var p1 = dbg.stopAt(testClass, testMethod);
var p2 = dbg.stopAt(testClass, failedLine);

>> Debugger Info: Try to connect to localhost:25432
>> Debugger Info: Successflly connected to localhost:25432
>> Debugger Info: Deferring breakpoint in org.apache.commons.lang3.math.NumberUtilsTest. It will be set after the class is loaded.
>> Debugger Info: Deferring breakpoint in org.apache.commons.lang3.math.NumberUtilsTest. It will be set after the class is loaded.


### Run the debugger & Check failed test

In [8]:
dbg.run();

>> Debugger Info: Debugger started.
>> Debugger Info: Breakpoint hit: line=1006, class=org.apache.commons.lang3.math.NumberUtilsTest, method=testIsNumber
>> Debugger Info: Current location,: line=1006, class=org.apache.commons.lang3.math.NumberUtilsTest, method=testIsNumber
1002 	     * Tests isNumber(String) and tests that createNumber(String) returns
1003 	     * a valid number iff isNumber(String) returns false.
1004 	     */
1005 	    public void testIsNumber() {
1006 =>	        String val = "12345";
1007 	        assertTrue("isNumber(String) 1 failed", NumberUtils.isNumber(val));
1008 	        assertTrue("isNumber(String)/createNumber(String) 1 failed", checkCreateNumber(val));
1009 	        val = "1234.5";
1010 	        assertTrue("isNumber(String) 2 failed", NumberUtils.isNumber(val));
1011 	        assertTrue("isNumber(String)/createNumber(String) 2 failed", checkCreateNumber(val));


[/Users/saku/Workspace/2020/JISDLab/debugspace/LoopN.java#L15](/tmp/lang_24_buggy/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java#L1006)

## Check failed line

In [9]:
dbg.cont();

>> Debugger Info: Breakpoint hit: line=1145, class=org.apache.commons.lang3.math.NumberUtilsTest, method=testIsNumber
>> Debugger Info: Current location,: line=1145, class=org.apache.commons.lang3.math.NumberUtilsTest, method=testIsNumber
1141 	        assertTrue("isNumber(String) LANG-521 failed", NumberUtils.isNumber(val));
1142 	
1143 	        // LANG-664
1144 	        val = "1.1L";
1145 =>	        assertFalse("isNumber(String) LANG-664 failed", NumberUtils.isNumber(val));
1146 	    }
1147 	
1148 	    private boolean checkCreateNumber(String val) {
1149 	        try {
1150 	            Object obj = NumberUtils.createNumber(val);


[/Users/saku/Workspace/2020/JISDLab/debugspace/HelloWorld.java#L10](/tmp/lang_24_buggy/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java#L1145)

## Start debugging

In [10]:
dbg.stepIn();

>> Debugger Info: Step completed: line=1320, class=org.apache.commons.lang3.math.NumberUtils, method=isNumber
>> Debugger Info: Current location,: line=1320, class=org.apache.commons.lang3.math.NumberUtils, method=isNumber
1316 	     * @param str  the <code>String</code> to check
1317 	     * @return <code>true</code> if the string is a correctly formatted number
1318 	     */
1319 	    public static boolean isNumber(String str) {
1320 =>	        if (StringUtils.isEmpty(str)) {
1321 	            return false;
1322 	        }
1323 	        char[] chars = str.toCharArray();
1324 	        int sz = chars.length;
1325 	        boolean hasExp = false;


[/Users/saku/Workspace/2020/JISDLab/debugspace/HelloWorld.java#L19](/tmp/lang_24_buggy/src/main/java/org/apache/commons/lang3/math/NumberUtils.java#L1320)

In [11]:
dbg.stopAt("org.apache.commons.lang3.math.NumberUtils", 1413)

Optional[jisd.debug.BreakPoint@ff2ef741]

In [12]:
dbg.cont();

>> Debugger Info: Breakpoint hit: line=1413, class=org.apache.commons.lang3.math.NumberUtils, method=isNumber
>> Debugger Info: Current location,: line=1413, class=org.apache.commons.lang3.math.NumberUtils, method=isNumber
1409 	            }
1410 	            if (chars[i] == 'l'
1411 	                || chars[i] == 'L') {
1412 	                // not allowing L with an exponent or decimal point
1413 =>	                return foundDigit && !hasExp && !hasDecPoint;
1414 	            }
1415 	            // last character is illegal
1416 	            return false;
1417 	        }
1418 	        // allowSigns is true iff the val ends in 'E'


[/Users/saku/Workspace/2020/JISDLab/debugspace/HelloWorld.java#L25](/tmp/lang_24_buggy/src/main/java/org/apache/commons/lang3/math/NumberUtils.java#L1413)

In [13]:
dbg.vars()

allowSigns=false
chars=instance of char[4] (id=484)
foundDigit=true
hasDecPoint=true
hasExp=false
i=3
start=0
str="1.1L"
sz=3
this.BYTE_MINUS_ONE=instance of java.lang.Byte(id=473)
this.BYTE_ONE=instance of java.lang.Byte(id=472)
this.BYTE_ZERO=instance of java.lang.Byte(id=470)
this.DOUBLE_MINUS_ONE=instance of java.lang.Double(id=478)
this.DOUBLE_ONE=instance of java.lang.Double(id=477)
this.DOUBLE_ZERO=instance of java.lang.Double(id=474)
this.FLOAT_MINUS_ONE=instance of java.lang.Float(id=483)
this.FLOAT_ONE=instance of java.lang.Float(id=482)
this.FLOAT_ZERO=instance of java.lang.Float(id=479)
this.INTEGER_MINUS_ONE=instance of java.lang.Integer(id=465)
this.INTEGER_ONE=instance of java.lang.Integer(id=464)
this.INTEGER_ZERO=instance of java.lang.Integer(id=455)
this.LONG_MINUS_ONE=instance of java.lang.Long(id=454)
this.LONG_ONE=instance of java.lang.Long(id=453)
this.LONG_ZERO=instance of java.lang.Long(id=449)
this.SHORT_MINUS_ONE=instance of java.lang.Short(id=469)
this.SHORT_

In [14]:
dbg.exit()

>> Debugger Info: Debugger exited.


##　Modify the source code and run the test again

In [15]:
Runnable diff = () -> exec(jisdCmd+" diff "+project);
Runnable test = () -> exec(jisdCmd+" test "+project);

In [16]:
diff.run();

diff --git a/org/apache/commons/lang3/math/NumberUtils.java b/org/apache/commons/lang3/math/NumberUtils.java
index 4a1d8459..d9b3c6de 100644
--- a/org/apache/commons/lang3/math/NumberUtils.java
+++ b/org/apache/commons/lang3/math/NumberUtils.java
@@ -1410,7 +1410,7 @@ public class NumberUtils {
             if (chars[i] == 'l'
                 || chars[i] == 'L') {
                 // not allowing L with an exponent or decimal point
-                return foundDigit && !hasExp;
+                return foundDigit && !hasExp && !hasDecPoint;
             }
             // last character is illegal
             return false;


In [17]:
test.run();

JUnit version 4.8.2
...................................................
Time: 0.032

OK (51 tests)


Running ant (compile)...................................................... OK
Running ant (compile.tests)................................................ OK


## Report a summary of the bug fix below
---

### Explanation of bug causes
小数点を含む文字列(例: 1.1L)が，数字として認識されている

### Explanation of bug fixes
文字列に小数点を含むかどうかのフラグのチェックを追加

---
